In [ ]:
import pandas as pd

# df = pd.read_csv('male transcription datasets/male_dataset_6.csv')
df = pd.read_excel('male transcription datasets/male_dataset_15.xlsx')

# Function to extract the last number from the old audio file path
def extract_last_number(path):
    # Split the path by underscores and extract the last part before the file extension
    return int(path.rsplit('_', 1)[-1].split('.')[0]) # split from the right once using the last _, get the last part, split again using . and get the first part
    # return int(path.rsplit('_', 2)[-2])

# Function to create the new audio file path
def create_new_path(old_path):
    last_number = extract_last_number(old_path)
    # Create the new path in the desired format
    # return f"female audio segments/female_audio_segments_1/1_{last_number}.mp3"
    return f"male audio segments/male_audio_segments_15/15_chunk_{last_number}.wav"

df['audio_path'] = df['Audio File'].apply(create_new_path)

df.to_excel('updated_male/updated_paths_m_15.xlsx', index=False)

print("Paths updated successfully!")

In [ ]:
import os

# List the folders
folder_paths = [
    ''
    
]

for folder_path in folder_paths:
    # Extract the folder number
    folder_number = folder_path.split('_')[-1]  # This will get the number at the end of the folder name
    
    files = os.listdir(folder_path)
    
    for file in files:
        if file.startswith('chunk'):
            # Construct the new file name with folder number prefix
            new_name = f"{folder_number}_{file}"
            
            # Get the full current and new paths
            current_path = os.path.join(folder_path, file)
            new_path = os.path.join(folder_path, new_name)
            
            # Rename the file
            os.rename(current_path, new_path)

    print(f"Renamed files in folder {folder_number} successfully!")

print("All files renamed successfully!")

In [ ]:
import os
import pandas as pd
from glob import glob
import re

folder_path = "updated_male"

data = []

xlsx_files = glob(os.path.join(folder_path, "*.xlsx")) # glob will search for files with the specified pattern and return them, updated/*.xlsx

for file in xlsx_files:
    df = pd.read_excel(file)

    if 'audio_path' in df.columns and 'Text (Arabic)' in df.columns:
        # Loop over each row and extract the relevant data
        for _, row in df.iterrows():
            audio_path = row['audio_path']
            audio_path = os.path.join(*audio_path.split(os.sep)[1:]) # split the path by the os seperator and then join the path from the second element, it is to remove the female audio segments thing first
            # Replace 'female_audio_segments_<number>' with 'audio'
            audio_path = re.sub(r'male_audio_segments_\d+/', 'audio/', audio_path) # replace the female_audio_segments_#/ with audio/
            transcript = row['Text (Arabic)']
            
            data.append({
                "audio": audio_path,
                "text": transcript
            })
            
df_data = pd.DataFrame(data)

df_data.to_csv("audio_transcripts.csv", index=False, encoding='utf-8-sig') #encoding to support arabic characters

print("CSV file saved as audio_transcripts.csv")


References:

-https://huggingface.co/docs/hub/en/datasets-adding

-https://huggingface.co/docs/hub/en/repositories-getting-started

-https://huggingface.co/docs/hub/en/datasets-data-files-configuration

-https://huggingface.co/docs/hub/en/datasets-audio
